## 使用 POST 方法來爬取台鐵的時刻表
### 本範例欲爬的目標網站：[列車時刻 / 車次查詢](https://www.railway.gov.tw/tra-tip-web/tip/tip001/tip112/gobytime)

In [ ]:
#
# 可以去掉 Python 輸出時，因為軟體版本所引起的警告的警告。
#
import warnings

warnings.filterwarnings('ignore')

In [ ]:
import requests

#
# 注意！注意！：執行本範例所選定的時刻表日期 (rideDate) 
#              一定要今天或之後才會有資料抓得下來
#
payload = {    
    '_csrf':'e7666daa-56a6-41b9-aba1-34141ed8e04b',
    'startStation':'0930-七堵',
    'endStation':'4220-臺南',
    'transfer':'ONE',
    'rideDate':'2025/07/01',
    'startOrEndTime':'true',
    'startTime':'00:00',
    'endTime':'23:59',
    'trainTypeList':'ALL',
    'query':'查詢'
}


#
# 使用者自訂 http browser request header 中的 user-agent 內容
#
headers = { 
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
             AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36'
          } 


url_address = 'https://www.railway.gov.tw/tra-tip-web/tip/tip001/tip112/querybytime'


res_post = requests.post( url_address, 
                          data = payload, 
                          headers = headers ) 


res_post.encoding = 'utf-8'  # 為了能夠順利讓網頁中的中文字正確的呈現出來

print(res_post.text)  # 看一下爬取下來網頁的內容

### **剖析網頁內容，瞭解要抓的表格在哪裡**

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(res_post.text, 'html.parser')

#
# 表格標籤特徵為 <table class="itinerary-controls">
#
tables = soup.select('table[class = "itinerary-controls"]')    # 寫法一

tables = soup.select('#pageContent > div > table.itinerary-controls')   # 寫法二 CSS Selecter

#tables = soup.select('#pageContent > div > table')   # 這個 CSS Selecter 寫法會抓不出正確的表格

print('\n共掃出 %d 個表格！\n' % len(tables))

### **用 Pandas 資料科學套件來讀取爬出的表格**

In [ ]:
!pip install  html5lib

In [ ]:
import pandas as pd

tables = pd.read_html(str(tables))  # 需要先裝 html5lib 套件

print('\n共掃出 %d 個表格！\n' % len(tables))

### **看一下前三個表格與最後倒數的三個表格**

In [ ]:
tables[0]   # 以本例來看，只有這一個表格是我們要的   

In [ ]:
tables[1]

In [ ]:
tables[2]

In [ ]:
tables[3]

In [ ]:
tables[-3]

In [ ]:
tables[-2]

In [ ]:
tables[-1]

### **看一下目標表格（時刻表）上有哪一些欄位**

In [ ]:
for name in tables[0].columns:
    
    print(name)

### **選擇特定欄位內的資料寫入最後結果的表格內**

In [ ]:
#
# 修改 Pandas 顯示設定
#
pd.set_option('display.max.columns', 20)


#
# 顯示全部
#
pd.set_option('display.max.rows', None)

In [ ]:
#column_fields = ['車種車次 (始發站 → 終點站)', '出發時間', '抵達時間', '行駛時間', '經由', '全票', '孩童票']

column_fields = tables[0].columns[:9]

df1 = pd.DataFrame( 
                   tables[0], 
                   columns = column_fields
                  ) 

#
# 調整最後表格的索引值由 1 開始，而不是依預設值從 0 開始 
#
df1.index += 1


#
# 顯示 df1 表格
#
df1  

### **抽出所要的表格內容**

In [ ]:
df2 = pd.DataFrame(columns = column_fields) 

# 
# 設定此例中要隔幾個 rows 來取車次的目標資料
#
jump_rows = 5


#
# 台鐵時刻表表格的最後一個索引值再加一
#
print('df1.index.stop = ', df1.index.stop)


for i in range(1, df1.index.stop, jump_rows):
    
    #
    # concate row(a series object) to the dataframe
    #    
   
    tmp = pd.DataFrame(df1.loc[i]).transpose()
    
    df2 = pd.concat( [df2, tmp], ignore_index = True)
    
    #
    # 或直接用以下的這一行替換上述的二行，但是 dataframe.append() 
    # 方法以後就會汰換掉，所以要改成 dataframe.concat()
    #
    #df2 = df2.append(df1.loc[i,:] , ignore_index = True)

    
#
# 調整最後表格的索引值由 1 開始，而不是依預設值從 0 開始    
#
df2.index += 1 

#df1.iloc[0, :]

#
# 顯示 df2 表格
#
df2  